In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

# 1.데이터 구조

spark는 RDD, Dataframe, Dataset 세가지 데이터 구조 제공

- RDD: 데이터가 **비구조적**인 경우 사용하기 적합, 모델 schema를 정하지 않고 사용 가능.
- Dataframe: 데이터가 schema와 datatype을 가진 **구조적**인 경우 사용.
- Spark의 RDD, Dataframe 모두 immutable이기 때문에 일단 생성되고 나면 원본 수정 불가.

| 데이터 구조 | 설명 |
|:--------|:--------|
| **RDD** | 비구조적, schema free, low-level |
| **Dataframe** | 구조적, schema 가짐 |  

------

# 2. RDD
- RDD
    - Resilient : 어느 한 노드에서 작업이 실패하면 다른 노드에서 실행
    - Distributed : 클러스터로 구성된 여러 노드에 분산해서 처리
    - Dataset : 데이터 구조
- 데이터를 저장하고 있는 Data Set
- 여러 컴퓨터에 분산해서 사용할 수 있다는 점이 특징
- RDD는 데이터가 비구조적인 경우 사용하기 적합
- 모델 schema를 정하지 않고 사용 가능
- RDD는 Python list, 파일, hdfs 등 다양한 자료에서 생성할 수 있고, 생성된 자료는 수정할 수 없는 read-only(immutable이기 때문)


## 2.1 RDD 생성하기
RDD는 sparkContext로부터 생성됨

| 생성 방법 | 설명 | 함수 |
|:--------|:--------|:--------|
| **내부에서 읽기** | Python list에서 생성 | `parallelize()` |
| **외부에서 읽기** | 파일, HDFS, HBase ... | `textFile("data/*txt")` |

### 2.1.1 list에서 RDD 생성하기

In [6]:
myList = [0,1,2,3,4,5,6,7,8,9]

In [7]:
myRDD = spark.sparkContext.parallelize(myList)

In [17]:
myRDD.first() # 첫번째 요소 보여줌

0

In [63]:
myRDD.count() # 요소 갯수 돌려줌

10

In [18]:
myRDD.take(3) # 앞에 요소 3개 list로 돌려줌

[0, 1, 2]

In [19]:
myRDD.collect() # 모든 요소 list로 돌려줌

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### 2.1.2 파일에서 RDD 생성하기

In [14]:
%%writefile data/file2RDD.txt
My name is Parkapark.
This is an jupyter notebook about Apache Spark.
Apahce Spark is an open source cluster computing framework.
Apache Spark Apache Spark Apache Spark Apache Spark
Written by Parkapark.

Writing data/file2RDD.txt


In [15]:
myRDD1 = spark.sparkContext.textFile(os.path.join("data/file2RDD.txt"))

In [20]:
myRDD1.first()

'My name is Parkapark.'

In [58]:
myRDD1.count()

5

In [21]:
myRDD1.take(3)

['My name is Parkapark.',
 'This is an jupyter notebook about Apache Spark.',
 'Apahce Spark is an open source cluster computing framework.']

In [23]:
myRDD1.collect()

['My name is Parkapark.',
 'This is an jupyter notebook about Apache Spark.',
 'Apahce Spark is an open source cluster computing framework.',
 'Apache Spark Apache Spark Apache Spark Apache Spark',
 'Written by Parkapark.']

### 2.1.3 CSV에서 RDD 생성하기

In [2]:
%%writefile data/CSV2RDD.csv
0, 0
1, 1
2, 2
3, 3
4, 4
5, 5
6, 6
7, 7
8, 8
9, 9

Overwriting data/CSV2RDD.csv


In [3]:
!cat data/CSV2RDD.csv

0, 0
1, 1
2, 2
3, 3
4, 4
5, 5
6, 6
7, 7
8, 8
9, 9


In [25]:
myRDD2 = spark.sparkContext.textFile(os.path.join("data/CSV2RDD.csv"))

In [26]:
myRDD2.first()

'0, 0'

In [59]:
myRDD2.count()

10

In [28]:
myRDD2.take(3)

['0, 0', '1, 1', '2, 2']

In [29]:
myRDD2.collect()

['0, 0',
 '1, 1',
 '2, 2',
 '3, 3',
 '4, 4',
 '5, 5',
 '6, 6',
 '7, 7',
 '8, 8',
 '9, 9']

In [30]:
myRDD3 = myRDD2.map(lambda line: line.split(','))

In [32]:
myRDD3.collect()

[['0', ' 0'],
 ['1', ' 1'],
 ['2', ' 2'],
 ['3', ' 3'],
 ['4', ' 4'],
 ['5', ' 5'],
 ['6', ' 6'],
 ['7', ' 7'],
 ['8', ' 8'],
 ['9', ' 9']]

## 2.2 RDD API
`데이터 변환` - transformation  
`연산` - action

### 2.2.1 데이터 변환
| 함수 | 설명 | 예제 |
|:--------|:--------|:--------|
| map(x) | 요소별로 x를 적용해서 결과 RDD 돌려줌 | .map(lambda x:x.split(',')) |
| filter(x) | 요소별로 선별하여 x를 적용해서 결과 RDD 돌려줌 | .filter(lambda x: "Spark" in x) |
| flatMap(x) | 요소별로 x를 적용하고, flat해서 결과 RDD 돌려줌 | .flatMap(lambda x: x.split(',')) |
| groupByKey() | key를 그룹해서 iterator를 돌려줌 |  |

#### 2.2.1.1 map()

In [39]:
sentence = "my name is park"
res = list(map(lambda x:x.split(),sentence))
print(res)

[['m'], ['y'], [], ['n'], ['a'], ['m'], ['e'], [], ['i'], ['s'], [], ['p'], ['a'], ['r'], ['k']]


In [40]:
sentence = ["my name is park","Here is PublicAI"]
res = list(map(lambda x:x.split(),sentence))
print(res)

[['my', 'name', 'is', 'park'], ['Here', 'is', 'PublicAI']]


#### 2.2.1.2 filter()

In [42]:
fibo = [0,1,1,2,3,5,8,13,21,34,55,89]
res = list(filter(lambda x: x % 2, fibo))
print(res)

[1, 1, 3, 5, 13, 21, 55, 89]


In [44]:
myRDD1.collect()

['My name is Parkapark.',
 'This is an jupyter notebook about Apache Spark.',
 'Apahce Spark is an open source cluster computing framework.',
 'Apache Spark Apache Spark Apache Spark Apache Spark',
 'Written by Parkapark.']

In [46]:
myRDD_parkapark = myRDD1.filter(lambda line: "Parkapark" in line)
print("Number of lines having 'Parkapark': ",myRDD_parkapark.count())

Number of lines having 'Parkapark':  2


#### 2.2.1.3 flatMap()

In [47]:
myRDD1.collect()

['My name is Parkapark.',
 'This is an jupyter notebook about Apache Spark.',
 'Apahce Spark is an open source cluster computing framework.',
 'Apache Spark Apache Spark Apache Spark Apache Spark',
 'Written by Parkapark.']

In [48]:
myRDD_flatmap = myRDD1.flatMap(lambda x: x.split())

In [51]:
myRDD_flatmap.collect()

['My',
 'name',
 'is',
 'Parkapark.',
 'This',
 'is',
 'an',
 'jupyter',
 'notebook',
 'about',
 'Apache',
 'Spark.',
 'Apahce',
 'Spark',
 'is',
 'an',
 'open',
 'source',
 'cluster',
 'computing',
 'framework.',
 'Apache',
 'Spark',
 'Apache',
 'Spark',
 'Apache',
 'Spark',
 'Apache',
 'Spark',
 'Written',
 'by',
 'Parkapark.']

#### 2.2.1.4 groupByKey()
- 자세히는 더 나중에 pairRDD에서 다룸

In [57]:
myRDD_flatmap.collect()

['My',
 'name',
 'is',
 'Parkapark.',
 'This',
 'is',
 'an',
 'jupyter',
 'notebook',
 'about',
 'Apache',
 'Spark.',
 'Apahce',
 'Spark',
 'is',
 'an',
 'open',
 'source',
 'cluster',
 'computing',
 'framework.',
 'Apache',
 'Spark',
 'Apache',
 'Spark',
 'Apache',
 'Spark',
 'Apache',
 'Spark',
 'Written',
 'by',
 'Parkapark.']

In [56]:
myRDD_flatmap\
.map(lambda x:(x,1))\
.groupByKey()\
.collect()

[('name', <pyspark.resultiterable.ResultIterable at 0x124578280>),
 ('is', <pyspark.resultiterable.ResultIterable at 0x1236b0760>),
 ('Parkapark.', <pyspark.resultiterable.ResultIterable at 0x124586940>),
 ('an', <pyspark.resultiterable.ResultIterable at 0x123a2c850>),
 ('jupyter', <pyspark.resultiterable.ResultIterable at 0x123cb9730>),
 ('Apache', <pyspark.resultiterable.ResultIterable at 0x123cb9640>),
 ('Spark.', <pyspark.resultiterable.ResultIterable at 0x124566c10>),
 ('Spark', <pyspark.resultiterable.ResultIterable at 0x124566e80>),
 ('open', <pyspark.resultiterable.ResultIterable at 0x124566d90>),
 ('source', <pyspark.resultiterable.ResultIterable at 0x124566220>),
 ('My', <pyspark.resultiterable.ResultIterable at 0x124566700>),
 ('This', <pyspark.resultiterable.ResultIterable at 0x124566460>),
 ('notebook', <pyspark.resultiterable.ResultIterable at 0x124566eb0>),
 ('about', <pyspark.resultiterable.ResultIterable at 0x1245667c0>),
 ('Apahce', <pyspark.resultiterable.ResultItera

#### 2.2.1.5 flatMap VS map  
- flatMap() 
    - 리스트 안에 또 리스트가 있는 경우 이를 하나의 리스트로 만듬
    - 모든 단어를 하나의 리스트로 만듬
- map()
    - 리스트 안에 또 리스트가 있는 구조를 보존하고 처리함  
    - 파일의 줄마다 리스트를 만듬
| 줄 | 원본 | flatMap() 결과 | map() 결과 |
|:--------|:--------|:--------|:--------|
| 1 | My name is | My name is | ['My','name','is'] |
| 2 | This is a jupyter | This is a jupyter | ['This','is','a','jupyter'] |

In [115]:
%%writefile data/file2RDD.txt
My name is Parkapark.
This is an jupyter notebook about Apache Spark.
Apahce Spark is an open source cluster computing framework.
Apache Spark Apache Spark Apache Spark Apache Spark
Written by Parkapark.

Overwriting data/file2RDD.txt


In [116]:
myRDD1 = spark.sparkContext.textFile(os.path.join("data/file2RDD.txt"))

**flatMap()**

In [125]:
myRDD1_flatmap = myRDD1.flatMap(lambda x:x.split(" ")).take(10)

In [126]:
myRDD1_flatmap

['My',
 'name',
 'is',
 'Parkapark.',
 'This',
 'is',
 'an',
 'jupyter',
 'notebook',
 'about']

In [127]:
for i in myRDD1_flatmap:
    print(i, end=' ')

My name is Parkapark. This is an jupyter notebook about 

**map()**

In [130]:
myRDD1_map = myRDD1.map(lambda x:x.split(" ")).take(2)

In [131]:
myRDD1_map

[['My', 'name', 'is', 'Parkapark.'],
 ['This', 'is', 'an', 'jupyter', 'notebook', 'about', 'Apache', 'Spark.']]

In [132]:
for i in myRDD1_map:
    print(i, end=' ')

['My', 'name', 'is', 'Parkapark.'] ['This', 'is', 'an', 'jupyter', 'notebook', 'about', 'Apache', 'Spark.'] 

### 2.2.2 연산 action
| 함수 | 설명 | 예제 |
|:--------|:--------|:--------|
| reduce(fn) | 요소별로 fn을 사용해서 줄여서 결과 list를 돌려줌 | .reduce(lambda x,y:x+y) |
| collect() | 모든 요소를 결과 list로 돌려줌 |  |
| count() | 요소 갯수를 결과 list로 돌려줌 |  |
| take(n) | collect()는 전체지만, n개만 돌려줌 | take(3) |
| countByKey() | key별 갯수를 세는 함수 | countByKey().items() |
| foreach(fn) | 각 데이터 항목에 함수 fn을 적용 |  |


#### 2.2.2.1 reduce()
reduce()는 lambda 함수를 사용해서 입력 데이터를 하나씩 서로 더해서 x+y 결과로 만들어 짐

In [88]:
myRDD_reduce = spark.sparkContext.parallelize(range(1,101))

In [89]:
myRDD_reduce.take(5)

[1, 2, 3, 4, 5]

In [90]:
myRDD_reduce.reduce(lambda x,y : x+y)

5050

#### 2.2.2.2 countByKey()

In [67]:
myRDD1.collect()

['My name is Parkapark.',
 'This is an jupyter notebook about Apache Spark.',
 'Apahce Spark is an open source cluster computing framework.',
 'Apache Spark Apache Spark Apache Spark Apache Spark',
 'Written by Parkapark.']

In [68]:
myRDD_flatmap = myRDD1.flatMap(lambda x: x.split())

In [83]:
myRDD_countByKey = myRDD_flatmap\
.map(lambda x:(x,1))\
.countByKey() # 결과값 dictionary로 출력

print(type(myRDD_countByKey))

<class 'collections.defaultdict'>


In [84]:
myRDD_countByKey

defaultdict(int,
            {'My': 1,
             'name': 1,
             'is': 3,
             'Parkapark.': 2,
             'This': 1,
             'an': 2,
             'jupyter': 1,
             'notebook': 1,
             'about': 1,
             'Apache': 5,
             'Spark.': 1,
             'Apahce': 1,
             'Spark': 5,
             'open': 1,
             'source': 1,
             'cluster': 1,
             'computing': 1,
             'framework.': 1,
             'Written': 1,
             'by': 1})

In [86]:
myRDD_countByKey = myRDD_flatmap\
.map(lambda x:(x,1))\
.countByKey()\
.items() # .items() 붙이면 리스트로 변환 가능

print(type(myRDD_countByKey))

<class 'dict_items'>


In [87]:
myRDD_countByKey

dict_items([('My', 1), ('name', 1), ('is', 3), ('Parkapark.', 2), ('This', 1), ('an', 2), ('jupyter', 1), ('notebook', 1), ('about', 1), ('Apache', 5), ('Spark.', 1), ('Apahce', 1), ('Spark', 5), ('open', 1), ('source', 1), ('cluster', 1), ('computing', 1), ('framework.', 1), ('Written', 1), ('by', 1)])

#### 2.2.2.3 foreach()
`foreach()`는 action이지만, 다른 action들과 다르게 반환 값이 없음
각 요소에 적용하는 역할에 대해 유사한 `map()`이 있으나 `map()`은 요소에 대해 계산을 하고, 그 값을 반환함

In [94]:
spark.sparkContext.parallelize([1, 2, 3, 4, 5]).foreach(lambda x: x + 1)

In [96]:
spark.sparkContext.parallelize([1, 2, 3, 4, 5]).map(lambda x: x + 1).collect()

[2, 3, 4, 5, 6]

In [97]:
def f(x): print(x)
spark.sparkContext.parallelize([1, 2, 3, 4, 5]).foreach(f)

<img width="570" alt="foreach" src="https://user-images.githubusercontent.com/71860179/98218847-9a090980-1f8f-11eb-82df-d770b2486308.png">


## 2.3 Pair RDD
- key,value 쌍으로 구성된 RDD
- key에 대해 연산을 하는 byKey() 또는 value에 대해  연산을 하는 byValue() 함수 사용 가능  


------
| 구분 | 예제 |
|:--------|:--------|
| groupByKey() | 같은 key를 grouping, 부분 partition에 먼저 reduce하지 않고, 전체로 계산한다. |
| reduceByKey() | 같은 key의 value를 합계, 부분 partition에서 먼저 reduce하고, 전체로 계산하다. grouping + aggregation. 즉 reduceByKey = groupByKey().reduce() |
| mapValues() | PairRDD는 key,value가 있기 마련이다. value에 대해 적용하는 함수이다. 즉 key가 아니라 value에 적용하는 함수이다. |
| combineByKey() | 키별로 합계, 개수 (key,(sum,count))를 계산 |

### 2.3.1 pair RDD 생성

partition 1,2,3로 데이터가 분할 되어 있다 가정

| P1 | P2 | P3 |
|:--------|:--------|:--------|
| (key1,1) | (key1,1) | (key1,1) |
| (key1,1) | (key2,1) | (key1,1) |
| (key1,1) |  | (key2,1) |
| (key2,1) |  | (key2,1) |
| (key2,1) |  |  |

In [145]:
testList=[("key1",1),("key1",1),("key1",1),("key2",1),("key2",1),
           ("key1",1),("key2",1),
           ("key1",1),("key1",1),("key2",1),("key2",1)]

In [146]:
testRDD = spark.sparkContext.parallelize(testList)

In [147]:
testRDD.getNumPartitions()

12

In [103]:
testRDD.collect()

[('key1', 1),
 ('key1', 1),
 ('key1', 1),
 ('key2', 1),
 ('key2', 1),
 ('key1', 1),
 ('key2', 1),
 ('key1', 1),
 ('key1', 1),
 ('key2', 1),
 ('key2', 1)]

In [104]:
testRDD.keys().collect()

['key1',
 'key1',
 'key1',
 'key2',
 'key2',
 'key1',
 'key2',
 'key1',
 'key1',
 'key2',
 'key2']

### 2.3.2 pair RDD - groupByKey()

In [106]:
testRDD.groupByKey().collect()

[('key1', <pyspark.resultiterable.ResultIterable at 0x124f3ce20>),
 ('key2', <pyspark.resultiterable.ResultIterable at 0x124efdaf0>)]

### 2.3.3 pair RDD - reduceByKey()
reduceByKey() == groupByKey().reduce(): 실제로 되지는 않고 의미상

In [105]:
testRDD.reduceByKey(lambda x,y:x+y).collect()

[('key1', 6), ('key2', 5)]

### 2.3.4 pair RDD - mapValues()

In [109]:
testRDD.mapValues(lambda x:x+1).collect()

[('key1', 2),
 ('key1', 2),
 ('key1', 2),
 ('key2', 2),
 ('key2', 2),
 ('key1', 2),
 ('key2', 2),
 ('key1', 2),
 ('key1', 2),
 ('key2', 2),
 ('key2', 2)]

### 2.3.5 pair RDD - combineByKey()
key 별로 `(key, (sum, count))`를 계산 함  

| 구분 | combiner | merge values | merge combiner |
|:--------|:--------|:--------|:--------|
| **설명** | 각 키에 대해 **(value,1)** 튜플 생성 | 값을 더해 나감 (sum,count), `sum+value`, `count+1` | partition 별로 conbiner를 더함 |


## 2.4 stopwords(불용어) 제거하고 word count 하기

### 2.4.0 데이터 쓰기, 불러오기

In [1]:
%%writefile data/spark_wiki.txt
Apache Spark is and open-source distributed general-purpose cluster-computing framework.
Spark provides an interface for programming entire clusters with implicit data parallelism and fault tolerance.
Originally developed at the University of California, Berkeley's AMP Lab, the Spark codebase was later donated to the Apache Software Foundation, which has maintained it since.
Apache Spark has its architectural foundation in the resilient distributed dataset(RDD), a read only multiset of data items distributed over a cluster of machines, that is maintained in a fault-tolerant way.
The Dataframe API was released as an abstraction on top of the RDD, followed by the Dataset API.
In Spark 1.x, the RDD was the primary application programming interface(API), but as of Spark 2.x use of the Dataset API is encouraged even though the RDD  API is not deprecated.
The RDD technology still underlies the Dataset API.


Overwriting data/spark_wiki.txt


In [2]:
myRDD_word_count = spark.sparkContext\
.textFile(os.path.join("data/spark_wiki.txt"))

In [3]:
myRDD_word_count.take(3)

['Apache Spark is and open-source distributed general-purpose cluster-computing framework.',
 'Spark provides an interface for programming entire clusters with implicit data parallelism and fault tolerance.',
 "Originally developed at the University of California, Berkeley's AMP Lab, the Spark codebase was later donated to the Apache Software Foundation, which has maintained it since."]

### 2.4.1 stopwords 불러오기

In [4]:
!python3 -m pip install nltk

In [38]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /Users/admin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

### 2.4.2 flatMap()

In [40]:
word_count = myRDD_word_count.flatMap(lambda x:x.split(" "))

In [41]:
word_count.take(10)

['Apache',
 'Spark',
 'is',
 'and',
 'open-source',
 'distributed',
 'general-purpose',
 'cluster-computing',
 'framework.',
 'Spark']

### 2.4.3 filter() 
- stopwords 리스트에 있으면 제거

In [42]:
word_count1 = word_count.filter(lambda x: x.lower() not in stop_words)

In [43]:
word_count1.take(10)

['Apache',
 'Spark',
 'open-source',
 'distributed',
 'general-purpose',
 'cluster-computing',
 'framework.',
 'Spark',
 'provides',
 'interface']

### 2.4.4 map()
- 단어별로 (x,1)로 구성

In [44]:
word_count2 = word_count1.map(lambda x:(x,1))

In [45]:
word_count2.take(10)

[('Apache', 1),
 ('Spark', 1),
 ('open-source', 1),
 ('distributed', 1),
 ('general-purpose', 1),
 ('cluster-computing', 1),
 ('framework.', 1),
 ('Spark', 1),
 ('provides', 1),
 ('interface', 1)]

### 2.4.5 reduceByKey()
- 동일한 단어의 value, 즉 갯수를 서로 합함

In [46]:
word_count3 = word_count2.reduceByKey(lambda x,y:x+y)

In [48]:
word_count3.take(10)

[('Apache', 3),
 ('Spark', 6),
 ('open-source', 1),
 ('general-purpose', 1),
 ('cluster-computing', 1),
 ('provides', 1),
 ('programming', 2),
 ('entire', 1),
 ('clusters', 1),
 ('implicit', 1)]

### 2.4.6 묶어서 하나로 많이 나온 단어 순으로 출력
- map(lambda x:(x[1],x[0])) : 내림차순 정렬하기 위해서 단어의 숫자 앞으로 오도록 x[1] 과 x[0] 바꾸기
- sortByKey(False) : 내림차순 정렬, default가 오름차순

In [55]:
word_count_final = (
    myRDD_word_count
    .flatMap(lambda x:x.split(' '))
    .filter(lambda x:x.lower() not in stop_words)
    .map(lambda x:(x,1))
    .reduceByKey(lambda x,y:x+y)
    .map(lambda x:(x[1],x[0]))
    .sortByKey(False) 
)

In [50]:
word_count_final.take(10)

[(6, 'Spark'),
 (3, 'Apache'),
 (3, 'distributed'),
 (3, 'API'),
 (3, 'Dataset'),
 (3, 'RDD'),
 (2, 'programming'),
 (2, 'data'),
 (2, 'maintained'),
 (2, 'API.')]